<a href="https://colab.research.google.com/github/MilindPius2005/deep-learning-notes/blob/main/gpu_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print("GPU Available:", tf.test.is_gpu_available())
print("GPU Name:", tf.config.list_physical_devices('GPU'))


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU Available: True
GPU Name: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:

tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.__version__


'2.19.0'

In [ ]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [ ]:
X_train.shape

(50000, 32, 32, 3)

In [ ]:
X_test.shape

(10000, 32, 32, 3)

In [ ]:
classes=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [ ]:
X_train_scaled=X_train/255
X_test_scaled=X_test/255

In [ ]:
y_train_categorical = keras.utils.to_categorical(y_train, num_classes=10)
y_test_categorical = keras.utils.to_categorical(y_test, num_classes=10)


In [ ]:

def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

making numpy array into tensorflow datasets

In [ ]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [ ]:
type(train_tf_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [ ]:
strategy= tf.distribute.MirroredStrategy()

In [ ]:
strategy.num_replicas_in_sync

1

In [ ]:
BATCH_SIZE_PER_REPLICA=250
BATCH_SIZE=BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
train_dataset=train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


USING DISTRIBUTED TRAINING

In [ ]:
with strategy.scope():
    gpu_model=get_model()
    gpu_model.fit(train_dataset,epochs=50)

Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.2532 - loss: 2.0936
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.3591 - loss: 1.8317
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.3835 - loss: 1.7610
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4012 - loss: 1.7149
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.4145 - loss: 1.6788
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4260 - loss: 1.6487
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4366 - loss: 1.6225
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4453 - loss: 1.5992
Epoch 9/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4528 - loss: 1.5785
Epoch 10/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4578 - loss: 1.5599
Epoch 11/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4635 - loss: 1.5429
Epoch 12/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 38s 177ms/step - accuracy: 0.2521 - loss: 2.0889
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 41s 181ms/step - accuracy: 0.3580 - loss: 1.8313
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 36s 179ms/step - accuracy: 0.3853 - loss: 1.7609
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 42s 182ms/step - accuracy: 0.4025 - loss: 1.7136
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 36s 182ms/step - accuracy: 0.4166 - loss: 1.6767
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 41s 184ms/step - accuracy: 0.4289 - loss: 1.6460
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 37s 184ms/step - accuracy: 0.4366 - loss: 1.6196
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 37s 184ms/step - accuracy: 0.4443 - loss: 1.5963
Epoch 9/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 41s 183ms/step - accuracy: 0.4514 - loss: 1.5755
Epoch 10/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 37s 185ms/step - accuracy: 0.4583 - loss: 1.5569
Epoch 11/50
 67/200 ━━━━━━━━━━━━━━━━━━━━ 24s 184ms/step - accuracy: 0.4638 - loss: 1.5335